In [16]:
import time
import random

class School(object):
    _teacher = None
    _student = None
    
    def __init__(self):
        # Start the run process everytime an instance is created.
        self._student = self.student()
        self._student.send(None)
        self._teacher = self.teacher()
        self._teacher.send(None)

    
    def forward(self):
        next(self._teacher)
        
    def terminate(self):
        """
        Terminate the school.
        """
        self._student.close()
        self._teacher.close()

    def teacher(self, interval = 3):
        print("Teacher starts waiting for the student to report")
        for count in range(5):
            yield 
            remark = self._student.send(random.randint(1, 5))
            print("Teacher report on student remark {}".format(remark))

    def student(self, interval=5):
        elapsed = 0
        print("Student starts reporting")
        while True:
            elapsed += interval
            
            value = (yield elapsed) 
            interval = value if value else interval
            print("New student report interval is {}".format(interval))
            
        
school = School()

Student starts reporting
Teacher starts waiting for the student to report


In [17]:
school.forward()

New student report interval is 2
Teacher report on student remark 7


In [30]:
import time
import random

class School(object):
    _teacher = None
    _student = None
    
    def __init__(self):
        # Start the run process everytime an instance is created.
        self._student = self.student()
        self._student.send(None)
        self._teacher = self.teacher()
        self._teacher.send(None)

    
    def forward(self, value=None):
        self._student.send(value)
        
    def terminate(self):
        """
        Terminate the school.
        """
        self._student.close()
        self._teacher.close()

    def teacher(self, interval = 3):
        print("Teacher starts waiting for the student to report")
        for count in range(5):
            student_report = yield 
            print("Teacher got a report from student: {}".format(student_report))

    def student(self):
        counts = 0
        print("Student starts reporting")
        while True:
            # Wait for the instruction to resume and report.
            value = (yield counts)

            # Report to the teacher
            self._teacher.send(counts)
            counts += 1

            
school = School()

Student starts reporting
Teacher starts waiting for the student to report


In [31]:
for i in range(3):
    school.forward()
    
school.terminate()
school.forward()

Teacher got a report from student: 0
Teacher got a report from student: 1
Teacher got a report from student: 2


StopIteration: 

# Car dispatching

In [26]:
from typing import Dict, Optional, Generator
import time

cycle = -1
env = None
def ev(env) -> Generator:
    next_charging_station = 9999999   
    while True:
        next_charging_station = (yield next_charging_station)
        print("ev: next_charging_station[{}]\n".format(next_charging_station))
        
def dispathcer(env, ev) -> Generator:
    global cycle

    station = 1
    while True:
        time.sleep(1)
        print('dispathcer: dispatching the next charging station {} to ev'.format(station))
        print("Unbloks ev to start the next cycle({}) with evaluating value=(yield ...)".format(cycle+1))
        print("The value is set in the stack-frame of cycle({}) and can be returned from yield in the cycle.\n".format(
            cycle+1
        ))
        value = ev.send(station)
        cycle +=1
        print("dispathcer: ev said the current station of the ev is [{}]".format(value))
        print("dispathcer: ev cycle({}) has ended blocking in yield statement.\n".format(cycle))
        station +=1
        
process_ev = ev(env)
print("The first send(None)/next() advances the execution in ev to yield at cycle({0}) which returns " +\
      "the value in the current cycle({0}) stack-frame. Ignore it returned: {1}.\n".format(
    cycle, process_ev.send(None)
))

print("ev is now blocking 'in' the yield statement BEFORE value=(yield ...) TO BE evaluated in the next cycle({}).\n".format(cycle+1))
process_dispatcher = dispathcer(env, process_ev)

The first send(None)/next() advances the execution in ev to yield at cycle({0}) which returns the value in the current cycle(-1) stack-frame. Ignore it returned: 9999999.

ev is now blocking 'in' the yield statement BEFORE value=(yield ...) TO BE evaluated in the next cycle(0).

dispathcer: dispatching the next charging station 1 to ev
Unbloks ev to start the next cycle(0) with evaluating value=(yield ...)
The value is set in the stack-frame of cycle(0) and can be returned from yield in the cycle.

ev: next_charging_station[1]

dispathcer: ev said the current station of the ev is [1]
dispathcer: ev cycle(0) has ended blocking in yield statement.

dispathcer: dispatching the next charging station 2 to ev
Unbloks ev to start the next cycle(1) with evaluating value=(yield ...)
The value is set in the stack-frame of cycle(1) and can be returned from yield in the cycle.

ev: next_charging_station[2]

dispathcer: ev said the current station of the ev is [2]
dispathcer: ev cycle(1) has ended 

KeyboardInterrupt: 